In [1]:
import sys
sys.path.append('./azureml')

In [2]:
from azureml.core import Model
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun
from fasttext_pipeline_utils import choose_workspace, deploy_locally, deploy_to_ACI, deploy_to_AKS

In [3]:
# chose a workspace
subscription_id = '4f455bd0-f95a-4b7d-8d08-078611508e0b'
resource_group = 'fundamental'
workspace_name = 'fundamental3'
# set this if you have multiple tenant
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
workspace=choose_workspace(subscription_id, resource_group, workspace_name, tenant_id)

name: fundamental3
resource_group fundamental
location eastasia
subscription_id 4f455bd0-f95a-4b7d-8d08-078611508e0b
compute_targets dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


In [4]:
# get a PipelineRun object
experiment_name = "deploy"
experiment = Experiment(workspace, experiment_name)
run_id = '24f0ba77-1ef8-4a9a-b9a8-9ed14d98190e'
pipeline_run = PipelineRun(experiment, run_id)

In [5]:
# get a StepRun object
step_run = pipeline_run.find_step_run(name = 'FastText Train')[0]

In [6]:
# check ports
print('input ports:')
input_ports = step_run.input_ports
for port in input_ports:
    print(port,'\n')
output_ports = step_run.output_ports
print('\n\noutput ports:')
for port in output_ports:
    print(port,'\n')

input ports:
Port(Name:Training data dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: 358a3e99-f299-4089-b2cf-cecc32ac34f8,
Type: azureml.StepRun,
Status: Completed)) 

Port(Name:Validation data dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: 358a3e99-f299-4089-b2cf-cecc32ac34f8,
Type: azureml.StepRun,
Status: Completed)) 

Port(Name:Char2index dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: 358a3e99-f299-4089-b2cf-cecc32ac34f8,
Type: azureml.StepRun,
Status: Completed)) 



output ports:
Port(Name:Trained model dir,
Type:['ModelDirectory'],
StepRun:Run(Experiment: deploy,
Id: 358a3e99-f299-4089-b2cf-cecc32ac34f8,
Type: azureml.StepRun,
Status: Completed)) 



In [7]:
# download model from the output port of the Train module
port = step_run.get_port(name='Trained model dir')
saved_path = port.download(local_path='data', overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir/BestModel, 1 files out of an estimated total of 1
model save at: data/azureml/358a3e99-f299-4089-b2cf-cecc32ac34f8/Trained_model_dir


In [8]:
# register model for deployment
model_name = os.listdir(saved_path)[0]
model_path = os.path.join(saved_path, model_name)
model = Model.register(workspace, model_path=model_path, model_name='model_for_deploy', tags={'deploy':1})

Registering model model_for_deploy


In [9]:
# register env if not registered before
name='env_for_deployment'
file_path='deployment/env_for_deployment.yaml'
env = Environment.from_conda_specification(name=name, file_path=file_path)
env = env.register(workspace=workspace)

In [10]:
# define inference configuration
entry_script='scoring_for_deployment.py'
version='1'
source_directory='deployment'
env = Environment.get(workspace=workspace, name=name, version=version)
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                       environment=env)

In [11]:
# deploy locally
service_name='local-deploy-test'
models=[model]
port=8892
service_locally = deploy_locally(workspace, service_name, models, inference_config, port=port)
print(service_locally.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry fundamental33c005c1f.azurecr.io
Logging into Docker registry fundamental33c005c1f.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM fundamental33c005c1f.azurecr.io/azureml/azureml_55f6443a7f1f616df548a00877130395
 ---> ecd09a31e78d
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 07267dc695a0
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjRmNDU1YmQwLWY5NWEtNGI3ZC04ZDA4LTA3ODYxMTUwOGUwYiIsInJlc291cmNlR3JvdXBOYW1lIjoiZnVuZGFtZW50YWwiLCJhY2NvdW50TmFtZSI6ImZ1bmRhbWVudGFsMyIsIndvcmtzcGFjZUlkIjoiYTdjMmFjYWEtYzhmMS00NDhiLWI4OTQtYzJlN2E3MWIzYTMyIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 17bfa353618e
 ---> 48ca5b5f9f65
Step 4/5 : RUN mv '/var/azureml-app/tmpnkpc7zl9.py' /var/azureml-app/main.py
 ---> Running in 3beed6280bdd
 ---> 9bbb238e3a44
Step 5/5 : CMD 

In [12]:
# deploy to ACI (Azure Container Instances)
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aci-deploy-test7'
models=[model]
service_aci=deploy_to_ACI(workspace, service_name, models, inference_config, cpu_cores=1, memory_gb=1, overwrite=True)

found existing service named aci-deploy-test7, delete it right now...
Running....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [13]:
# deploy to AKS (Azure Kubernetes Service)
# workspace with AKS
subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
resource_group = 'DesignerDRI'
workspace_name = 'DesignerDRI_EASTUS'
# set this if you have multiple tenant
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"

workspace_aks=choose_workspace(subscription_id, resource_group, workspace_name, tenant_id)

# register model in this workspace
model = Model.register(workspace_aks, model_path=model_path, model_name='model_for_deploy', tags={'deploy':1})

name: DesignerDRI_EASTUS
resource_group DesignerDRI
location eastus
subscription_id 74eccef0-4b8d-4f83-b5f9-fa100d155b22
compute_targets dict_keys(['attachedcompute', 'myaks1', 'default', 'compute', 'cpu-cluster', 'aml-compute'])
Registering model model_for_deploy


In [14]:
# deploy to AKS (Azure Kubernetes Service)
attachment_name='myaks1'
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aks-deploy-test7'
models=[model]
token_auth_enabled=True
service_aks=deploy_to_AKS(workspace_aks, attachment_name, service_name, models, inference_config, token_auth_enabled=token_auth_enabled,
                  cpu_cores=1, memory_gb=1)

found existing service named aks-deploy-test7, delete it right now...
auth type: token
Running............................................................
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    },
    {
      "code": "DeploymentFailed",
      "message": "Couldn't schedule because the kubernetes cluster didn't have available resources after trying for 00:05:00.\nYou can address this error by either adding more nodes, changing the SKU of your nodes or changing the resource requirements of your service.\nPlease refer to https://aka.ms/debugimage#container-cannot-be-scheduled for more information."
    }
  ]
}

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More informa

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 79909e99-c1db-4574-95bc-edb3d83ce7d8
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    },
    {
      "code": "DeploymentFailed",
      "message": "Couldn't schedule because the kubernetes cluster didn't have available resources after trying for 00:05:00.\nYou can address this error by either adding more nodes, changing the SKU of your nodes or changing the resource requirements of your service.\nPlease refer to https://aka.ms/debugimage#container-cannot-be-scheduled for more information."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached

In [ ]:
# consume service
# consume deployment
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header
# Add content type header
headers.update({'Content-Type':'application/json'})
# print(headers)


standard_sample_input = {'param':{'input_sentence': '受疫情影响, 今年很多学生不得不在家上课'}}
standard_sample_input = json.dumps(standard_sample_input)

service = service_locally
# service = service_aci

# service = service_aks
# token, refresh_by = service.get_token()
# headers['Authorization']=f'Bearer {token}'


response = requests.post(service.scoring_uri, data=standard_sample_input, headers=headers)
print(service.scoring_uri)
print(response)
# print(response.status_code)
# print(response.elapsed)
print(response.content)
print(response.json())